***vis_videos_seen_skipped* function**:
* Input: players_info dictionary
* Displays bar chart showing for each video the total number of times it has been seen and skipped

In [ ]:
def vis_videos_seen_skipped (players_info):
    videos_seen = {}
    videos_skipped = {}

    for player in players_info.keys():
        # videos seen
        for video_seen in players_info[player]["videos_seen"]:
            if video_seen in videos_seen.keys():
                videos_seen[video_seen] += 1
            else:
                videos_seen[video_seen] = 1
            if video_seen not in videos_skipped.keys():
                videos_skipped[video_seen] = 0
        # videos skipped
        for video_skipped in players_info[player]["videos_skipped"]:
            if video_skipped in videos_skipped.keys():
                videos_skipped[video_skipped] += 1
            else:
                videos_skipped[video_skipped] = 1
            if video_skipped not in videos_seen.keys():
                videos_seen[video_skipped] = 0
            else:
                videos_seen[video_skipped] -= 1
                
    # Bar chart showing for each video
    # the total number of times it has been seen and skipped

    N = len(videos_seen)
    ind = np.arange(N)

    width = 0.35
    if N>0:
        p1 = plt.bar(ind, videos_seen.values(), width, color='orange', label='Seen')
        p2 = plt.bar(ind, videos_skipped.values(), width, color='lightblue', label='Seen but skipped', bottom=list(videos_seen.values()))
        plt.xlabel("Video")
        plt.ylabel("Number of times")
        plt.xticks(ind, videos_seen.keys(),rotation=90)
        plt.legend()
        plt.title("Videos seen and skipped by players")
        plt.show()